# Rainfall Forecasting: Exploring Weather Predictions with Machine Learning

# About The Dataset


The original source of the data is Australian Government's Bureau of Meteorology and the latest data can be gathered from [http://www.bom.gov.au/climate/dwo/](http://www.bom.gov.au/climate/dwo/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01).

The dataset to be used has extra columns like 'RainToday' and our target is 'RainTomorrow', which was gathered from the Rattle at [https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData](https://bitbucket.org/kayontoga/rattle/src/master/data/weatherAUS.RData?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)




This dataset contains observations of weather metrics for each day from 2008 to 2017. The **weatherAUS.csv** dataset includes the following fields:

| Field         | Description                                           | Unit            | Type   |
| ------------- | ----------------------------------------------------- | --------------- | ------ |
| Date          | Date of the Observation in YYYY-MM-DD                 | Date            | object |
| Location      | Location of the Observation                           | Location        | object |
| MinTemp       | Minimum temperature                                   | Celsius         | float  |
| MaxTemp       | Maximum temperature                                   | Celsius         | float  |
| Rainfall      | Amount of rainfall                                    | Millimeters     | float  |
| Evaporation   | Amount of evaporation                                 | Millimeters     | float  |
| Sunshine      | Amount of bright sunshine                             | hours           | float  |
| WindGustDir   | Direction of the strongest gust                       | Compass Points  | object |
| WindGustSpeed | Speed of the strongest gust                           | Kilometers/Hour | object |
| WindDir9am    | Wind direction averaged of 10 minutes prior to 9am    | Compass Points  | object |
| WindDir3pm    | Wind direction averaged of 10 minutes prior to 3pm    | Compass Points  | object |
| WindSpeed9am  | Wind speed averaged of 10 minutes prior to 9am        | Kilometers/Hour | float  |
| WindSpeed3pm  | Wind speed averaged of 10 minutes prior to 3pm        | Kilometers/Hour | float  |
| Humidity9am   | Humidity at 9am                                       | Percent         | float  |
| Humidity3pm   | Humidity at 3pm                                       | Percent         | float  |
| Pressure9am   | Atmospheric pressure reduced to mean sea level at 9am | Hectopascal     | float  |
| Pressure3pm   | Atmospheric pressure reduced to mean sea level at 3pm | Hectopascal     | float  |
| Cloud9am      | Fraction of the sky obscured by cloud at 9am          | Eights          | float  |
| Cloud3pm      | Fraction of the sky obscured by cloud at 3pm          | Eights          | float  |
| Temp9am       | Temperature at 9am                                    | Celsius         | float  |
| Temp3pm       | Temperature at 3pm                                    | Celsius         | float  |
| RainToday     | If there was rain today                               | Yes/No          | object |
| RainTomorrow  | If there is rain tomorrow                             | Yes/No          | float  |

Column definitions were gathered from [http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml](http://www.bom.gov.au/climate/dwo/IDCJDW0000.shtml?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01)



## **Import the required libraries**


In [ ]:
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

In [1]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
#if you are running the lab in your  browser-
import piplite
await piplite.install(['pandas'])
await piplite.install(['numpy'])


In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

### Importing the Dataset


In [4]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [5]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [6]:
await download(path, "Weather_Data.csv")
filename ="Weather_Data.csv"

In [7]:
df = pd.read_csv("Weather_Data.csv")
df.head()

Date  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine WindGustDir  \
0  2/1/2008     19.5     22.4      15.6          6.2       0.0           W   
1  2/2/2008     19.5     25.6       6.0          3.4       2.7           W   
2  2/3/2008     21.6     24.5       6.6          2.4       0.1           W   
3  2/4/2008     20.2     22.8      18.8          2.2       0.0           W   
4  2/5/2008     19.7     25.7      77.4          4.8       0.0           W   

   WindGustSpeed WindDir9am WindDir3pm  ...  Humidity9am  Humidity3pm  \
0             41          S        SSW  ...           92           84   
1             41          W          E  ...           83           73   
2             41        ESE        ESE  ...           88           86   
3             41        NNE          E  ...           83           90   
4             41        NNE          W  ...           88           74   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1017.6       1017.4         8         8     20.7     20.9        Yes   
1       1017.9       1016.4         7         7     22.4     24.8        Yes   
2       1016.7       1015.6         7         8     23.5     23.0        Yes   
3       1014.2       1011.8         8         8     21.4     20.9        Yes   
4       1008.3       1004.8         8         8     22.5     25.5        Yes   

   RainTomorrow  
0           Yes  
1           Yes  
2           Yes  
3           Yes  
4           Yes  

[5 rows x 22 columns]

### Data Preprocessing


#### One Hot Encoding


First, to perform one hot encoding to convert categorical variables to binary variables.


In [8]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

 The values of the 'RainTomorrow' column changing them from a categorical column to a binary column. We do not use the `get_dummies` method because we would end up with two columns for 'RainTomorrow' and we do not want, since 'RainTomorrow' is our target.


In [9]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

### Training Data and Test Data


In [10]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [11]:
df_sydney_processed = df_sydney_processed.astype(float)

In [12]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

### Linear Regression


### Using the `train_test_split` function to split the `features` and `Y` dataframes with a `test_size` of `0.2` and the `random_state` set to `10`.


In [16]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

### Creating and training a Linear Regression model called LinearReg using the training data (`x_train`, `y_train`).


In [18]:
LinearReg = LinearRegression().fit(x_train, y_train)

Now using the `predict` method on the testing data (`x_test`) and saving it to the array `predictions`.


In [24]:
predictions = LinearReg.predict(x_test)
print(predictions[0:3])

[0.13180542 0.27623749 0.97826004]


### Using the `predictions` and the `y_test` dataframe calculating the value for each metric using the appropriate function.


In [25]:
LinearRegression_MAE = sum(abs(y_test - predictions)) / len(y_test)
LinearRegression_MSE = sum((y_test - predictions) ** 2) / len(y_test)

y_mean = sum(y_test) / len(y_test)
ss_total = sum((y_test - y_mean) ** 2)
ss_residual = sum((y_test - predictions) ** 2)
LinearRegression_R2 =  1 - (ss_residual / ss_total) 

Mean Absolute Error (MAE): 0.2563193284828244
Mean Squared Error (MSE): 0.1157212427649327
R-squared (R²): 0.42712884038094456


### Showing the MAE, MSE, and R2 in a tabular format using data frame for the linear model.


In [28]:
Report = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'R²'],
    'Value': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})

print(Report)

  Metric     Value
0    MAE  0.256319
1    MSE  0.115721
2     R²  0.427129


### KNN


### Creating and training a KNN model called KNN using the training data (`x_train`, `y_train`) with the `n_neighbors` parameter set to `4`.


In [29]:
KNN = KNeighborsClassifier(n_neighbors=4).fit(x_train, y_train)

### Now using the `predict` method on the testing data (`x_test`) and saving it to the array `predictions`.


In [44]:
 predictions = KNN.predict(x_test)

[0. 0. 1. 0. 0. 0. 0. 1. 1. 0.]


### Using the `predictions` and the `y_test` dataframe calculating the value for each metric using the appropriate function.


In [64]:
KNN_Accuracy_Score = accuracy_score(y_test, predictions)
KNN_JaccardIndex = jaccard_score(y_test, predictions)
KNN_F1_Score = f1_score(y_test, predictions)

KNN Accuracy Score: 0.8366412213740458
KNN Jaccard Index: 0.5091743119266054
KNN F1 Score: 0.6747720364741641


### Decision Tree


####  Creating and training a Decision Tree model called Tree using the training data (`x_train`, `y_train`).


In [45]:
Tree = DecisionTreeClassifier().fit(x_train, y_train)

### Now using `predict` method on the testing data (`x_test`) and saving it to the array `predictions`.


In [47]:
predictions = Tree.predict(x_test)

[0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0.]


### Using the `predictions` and the `y_test` dataframe calculating the value for each metric using the appropriate function.


In [63]:
Tree_Accuracy_Score = accuracy_score(y_test, predictions)
Tree_JaccardIndex = jaccard_score(y_test, predictions)
Tree_F1_Score = f1_score(y_test, predictions)

Decision Tree Accuracy Score: 0.8366412213740458
Decision Tree Jaccard Index: 0.5091743119266054
Decision Tree F1 Score: 0.6747720364741641


### Logistic Regression


### Using `train_test_split` function to split the `features` and `Y` dataframes with a `test_size` of `0.2` and the `random_state` set to `1`.


In [49]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=1)

### Creating and training a LogisticRegression model called LR using the training data (`x_train`, `y_train`) with the `solver` parameter set to `liblinear`.


In [51]:
LR = LogisticRegression(solver='liblinear').fit(x_train, y_train)

LogisticRegression(solver='liblinear')

In [54]:
predictions = LR.predict(x_test)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [58]:
predict_proba = LR.predict_proba(x_test)

[[0.74010003 0.25989997]
 [0.97550665 0.02449335]
 [0.52853719 0.47146281]
 [0.85318912 0.14681088]]


 ### Using the `predictions`, `predict_proba` and the `y_test` dataframe calculatimg the value for each metric using the appropriate function.


In [65]:
LR_Accuracy_Score = accuracy_score(y_test, predictions)
LR_JaccardIndex = jaccard_score(y_test, predictions)
LR_F1_Score = f1_score(y_test, predictions)
LR_Log_Loss = log_loss(y_test, predict_proba)

Logistic Regression Accuracy Score: 0.8366412213740458
Logistic Regression Jaccard Index: 0.5091743119266054
Logistic Regression F1 Score: 0.6747720364741641
Logistic Regression Log Loss: 0.381414506461098


### SVM


 Creating and training a SVM model called SVM using the training data (`x_train`, `y_train`).


In [69]:
SVM = svm.SVC().fit(x_train, y_train)


SVC()

In [73]:
predictions = SVM.predict(x_test)



array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [75]:
SVM_Accuracy_Score = accuracy_score(y_test, predictions)
SVM_JaccardIndex = jaccard_score(y_test, predictions,average='weighted')
SVM_F1_Score = f1_score(y_test, predictions,average='weighted')

SVM Accuracy Score: 0.7221374045801526
SVM Jaccard Index: 0.5214824310937591
SVM F1 Score: 0.6056223268908018


### Report


 Showing the Accuracy,Jaccard Index,F1-Score and LogLoss in a tabular format using data frame for all of the above models.



In [79]:
Report =  {
    'Model': ['Logistic Regression', 'K-Nearest Neighbors', 'Support Vector Machine', 'Decision Tree'],
    'Accuracy Score': [LR_Accuracy_Score, KNN_Accuracy_Score, SVM_Accuracy_Score, Tree_Accuracy_Score],
    'Jaccard Index': [LR_JaccardIndex, KNN_JaccardIndex, SVM_JaccardIndex, Tree_JaccardIndex],
    'F1-Score': [LR_F1_Score, KNN_F1_Score, SVM_F1_Score, Tree_F1_Score],
    'Log Loss': [LR_Log_Loss, None, None, None]
}
Report = pd.DataFrame(Report)
print(Report)

Model  Accuracy Score  Jaccard Index  F1 Score  Log Loss
0    Linear Regression        0.256319       0.000000  0.000000  0.000000
1                  KNN        0.836641       0.509174  0.674772  0.000000
2        Decision Tree        0.836641       0.509174  0.674772  0.000000
3  Logistic Regression        0.836641       0.509174  0.674772  0.381415
4                  SVM        0.722137       0.521482  0.605622  0.000000